In [1]:
# !pip3 install sagemaker boto3

In [1]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json
#role = 'arn:aws-cn:iam::969422986683:role/AmazonSageMaker-ExecutionRole-20200517T121567'
role = sagemaker.get_execution_role()   # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

# inference_image_uri = (
#     f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

inference_image_uri = (
    f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
)
!rm model.tar.gz
!tar czvf model.tar.gz Buffer_instruct_deploy_code

s3_code_prefix ='s3://sagemaker-cn-north-1-020250557107/buffer-instruct-InternLM-code'
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")


Buffer_instruct_deploy_code/
Buffer_instruct_deploy_code/serving.properties
Buffer_instruct_deploy_code/model.py
Buffer_instruct_deploy_code/.ipynb_checkpoints/
Buffer_instruct_deploy_code/.ipynb_checkpoints/serving-checkpoint.properties
Buffer_instruct_deploy_code/requirements.txt
S3 Code or Model tar ball uploaded to --- > s3://sagemaker-cn-north-1-020250557107/s3://sagemaker-cn-north-1-020250557107/buffer-instruct-InternLM-code/model.tar.gz


In [2]:
from sagemaker.utils import name_from_base
import boto3

model_name = "pytorch-inference-llm-v1" # Append a timestamp to the provided string
endpoint_config_name = f"{model_name}-config"

endpoint_name = model_name

import boto3

client = boto3.client('sagemaker')
try:
    response = client.delete_model(ModelName=model_name)
    response = client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
except:
    pass

print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")



endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.4xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
# endpoint_config_response

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)


pytorch-inference-llm-v1
Image going to be used is ---- > 727897471807.dkr.ecr.cn-north-1.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117
Created Model: arn:aws-cn:sagemaker:cn-north-1:020250557107:model/pytorch-inference-llm-v1
Created Endpoint: arn:aws-cn:sagemaker:cn-north-1:020250557107:endpoint/pytorch-inference-llm-v1
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws-cn:sagemaker:cn-north-1:020250557107:endpoint/pytorch-inference-llm-v1
Status: InService


In [3]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")


question1 = "请问AWS Clean Rooms是多方都会收费吗？"
context1 = "Q: 是否发起者和数据贡献者都会被收费？\nA: 是单方收费，只有查询的接收方会收费"
prompts_template = """{system_role_prompt}，以下context内的文本内容为背景知识：\n<context>\n{context}\n</context>\n请根据背景知识, 回答这个问题：{question}"""
prompt = prompts_template.format(system_role_prompt="你是云服务AWS的智能客服机器人AWSBot", context=context1, question=question1)

response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "ask": prompt,
                "temperature": 0.01,
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

CPU times: user 23.1 ms, sys: 3.82 ms, total: 26.9 ms
Wall time: 2.29 s


'{\n  "answer":"不是的,只有发起者会被收费。"\n}'

'{\n  "outputs":"不是。AWS Clean Rooms是单方收费,只有查询的接收方会收费。",\n  "history":[\n    [\n      "你是云服务AWS的智能客服机器人AWSBot，以下context内的文本内容为背景知识：\\n<context>\\nQ: 是否发起者和数据贡献者都会被收费？\\nA: 是单方收费，只有查询的接收方会收费\\n</context>\\n请根据背景知识, 回答这个问题：请问AWS Clean Rooms是多方都会收费吗？",\n      "不是。AWS Clean Rooms是单方收费,只有查询的接收方会收费。"\n    ]\n  ]\n}'

In [ ]:
!aws sagemaker delete-endpoint --endpoint-name buffer-instruct-baichuan-001-2023-07-20-05-11-49-585-endpoint

In [ ]:
!aws sagemaker delete-endpoint-config --endpoint-config-name buffer-instruct-baichuan-001-2023-07-20-05-11-49-585-config

In [ ]:
!aws sagemaker delete-model --model-name buffer-instruct-baichuan-001-2023-07-20-05-11-49-585